# teex

### Evaluation of explanation quality: feature importance vectors

In this notebook, we are going to explore how we can use **teex** to evaluate feature importance explanations

In [3]:
from teex.featureImportance.data import SenecaFI, lime_to_feature_importance, scale_fi_bounds
from teex.featureImportance.eval import feature_importance_scores

from lime.lime_tabular import LimeTabularExplainer

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import fbeta_score

We are going to 

1. Generate synthetic data with feature importance explanations
2. Train a black box model on the data
3. Generate LIME explanations of the data
4. Evaluate the LIME explanations agains the ground truth

1. Generating the data

In [4]:
dataGen = SenecaFI(nSamples=1000, nFeatures=4, randomState=0)

X, y, exps = dataGen[:]

In [5]:
exps[0]

array([-0.6814,  0.9974,  0.2436,  0.9975], dtype=float32)

2. Training a black box model on the data

In [16]:
# split data 
Xtr, Xte = X[:600], X[600:]
ytr, yte = y[:600], y[600:]

In [17]:


model = RandomForestClassifier()
model.fit(Xtr, ytr)

RandomForestClassifier()

In [20]:
# F1 Score
print('Train F1: ', fbeta_score(model.predict(Xtr), ytr, beta=1))
print('Test F1: ', fbeta_score(model.predict(Xte), yte, beta=1))

Train F1:  1.0
Test F1:  0.941358024691358


The classifier fits the data quite nicely.

3. Generating LIME explanations

We first instance the explainer

In [41]:
explainer = LimeTabularExplainer(Xtr, feature_names=dataGen.featureNames, mode='classification')

# sample explanation
explainer.explain_instance(Xtr[0], model.predict_proba, mode='classification').show_in_notebook()

and generate the explanations. Unfortunately, with LIME we have to generate explanations 1 by 1. We use a function for transforming LIME explanations into feature importance vectors. This function does not transform the data, so we also import the feature importance scaler utility function, which will scale each feature to the (-1, 1) or (0, 1) range.

In [26]:
limeExps = []

for testObs in Xte:
    exp = explainer.explain_instance(testObs, model.predict_proba, mode='classification')
    limeExps.append(lime_to_feature_importance(exp, nFeatures=4))
    
limeExps = np.array(limeExps)

In [ ]:
limeExps[:5]

4. Evaluating LIME explanations

<p float="left">
  <img src="https://taiao.ai/img/6825_TAIAO_logo_1000x320.jpg" alt="drawing" style="width:150px;"/>
  <img src="https://www.bourses-etudiants.ma/wp-content/uploads/2018/06/University-of-Waikato-logo.png" alt="drawing" style="width:150px;"/>
  <img src="https://www.upc.edu/comunicacio/ca/identitat/descarrega-arxius-grafics/fitxers-marca-principal/upc-positiu-p3005.png" alt="drawing" style="width:200px;"/>
</p>